In [1]:
#from tensorflow_docs.vis import embed
from tensorflow import keras
#from imutils import paths

#import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
#import imageio
import cv2
from ipywidgets import Video, Image

import os
import json

Label preparation

In [2]:
data_df = pd. read_csv('/home/app/src/data/CSV/dataset_df.csv', dtype={"shot": str})

data_df.drop(columns=['scale_val'], inplace=True)
data_df.drop(columns = ['move_val'], inplace=True)

train_path = '/workspaces/final-project-shot-type/data/shot-type-dataset/trailer_v3/train/'
data_df['video_name'] = train_path + 'shot_' + data_df['movie'] + '_' + data_df['shot'] + '.mp4'
train_df = data_df[(data_df['dataset']=='train')]
train_df = train_df[['video_name', 'move_label','scale_label']]
train_df = train_df.rename(columns ={'move_label': 'tag'})


test_path = '/workspaces/final-project-shot-type/data/shot-type-dataset/trailer_v3/test/'
data_df['video_name'] = test_path + 'shot_' + data_df['movie'] + '_' + data_df['shot'] + '.mp4'
test_df = data_df[data_df['dataset']=='test']
test_df = test_df[['video_name', 'move_label','scale_label']]
test_df = test_df.rename(columns ={'move_label': 'tag'})

In [3]:
train_df

,video_name,tag,scale_label
0,/workspaces/final-project-shot-type/data/shot-...,Static,CS
1,/workspaces/final-project-shot-type/data/shot-...,Static,CS
2,/workspaces/final-project-shot-type/data/shot-...,Static,ECS
3,/workspaces/final-project-shot-type/data/shot-...,Static,MS
4,/workspaces/final-project-shot-type/data/shot-...,Static,CS
...,...,...,...
20851,/workspaces/final-project-shot-type/data/shot-...,Push,FS
20852,/workspaces/final-project-shot-type/data/shot-...,Motion,LS
20853,/workspaces/final-project-shot-type/data/shot-...,Push,LS
20854,/workspaces/final-project-shot-type/data/shot-...,Motion,FS


In [4]:
test_df

,video_name,tag,scale_label
25466,/workspaces/final-project-shot-type/data/shot-...,Static,LS
25467,/workspaces/final-project-shot-type/data/shot-...,Static,MS
25468,/workspaces/final-project-shot-type/data/shot-...,Static,MS
25469,/workspaces/final-project-shot-type/data/shot-...,Static,LS
25470,/workspaces/final-project-shot-type/data/shot-...,Motion,FS
...,...,...,...
33648,/workspaces/final-project-shot-type/data/shot-...,Static,FS
33649,/workspaces/final-project-shot-type/data/shot-...,Motion,MS
33650,/workspaces/final-project-shot-type/data/shot-...,Motion,FS
33651,/workspaces/final-project-shot-type/data/shot-...,Static,LS


In [5]:
# train_df.to_csv('/home/app/src/data/CSV/dataset_train_20856_train_test_8187.csv', index = False)
# test_df.to_csv('/home/app/src/data/CSV/dataset_test_20856_train_test_8187.csv', index = False)

In [6]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

2022-11-14 21:50:34.809931: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 21:50:34.818019: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 21:50:34.818870: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 21:50:34.820516: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

['Motion', 'Multi_movement', 'Pull', 'Push', 'Static']


 SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 21:50:35.501833: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10794 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7


In [7]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    return labels

train_labels = prepare_all_videos(train_df, "train")
test_labels = prepare_all_videos(test_df, "test")


In [8]:
# path_load_embeddings= '/home/app/src/embeddings/mov_20856_train_8187_test_emb/'

# #Train
# train_data_0 = np.load(path_load_embeddings + 'train_data_embedding_0_10k.npy')
# train_data_1 = np.load(path_load_embeddings + 'train_data_embedding_1_10k.npy')
# train_data=(train_data_0, train_data_1)
# #train_labels = np.load(path_load_embeddings + 'train_labels_embedding.npy')

# #Test
# test_data_0 = np.load(path_load_embeddings + 'test_data_embedding_0_10k.npy')
# test_data_1 = np.load(path_load_embeddings + 'test_data_embedding_1_10k.npy')
# test_data=(test_data_0, test_data_1)
# #test_labels = np.load(path_load_embeddings + 'test_labels_embedding.npy')

In [9]:
path_save_embeddings= '/home/app/src/embeddings/mov_20856_train_8187_test_emb/20856_train_8187_test_10_frames/'

# Train
#train_data_embedding_0 = np.save(path_save_embeddings + 'train_data_embedding_0.npy', train_data[0])
#train_data_embedding_1 = np.save(path_save_embeddings + 'train_data_embedding_1.npy', train_data[1])
train_labels_embedding = np.save(path_save_embeddings + 'train_labels_embedding.npy', train_labels)

PermissionError: [Errno 13] Permission denied: '/home/app/src/embeddings/mov_20856_train_8187_test_emb/20856_train_8187_test_10_frames/train_labels_embedding.npy'

In [ ]:
#Test
#test_data_embedding_0 = np.save(path_save_embeddings + 'test_data_embedding_0.npy', test_data[0])
#test_data_embedding_1 = np.save(path_save_embeddings + 'test_data_embedding_1.npy', test_data[1])
test_labels_embedding = np.save(path_save_embeddings + 'test_labels_embedding.npy', test_labels)

PermissionError: [Errno 13] Permission denied: '/home/app/src/embeddings/mov_20856_train_8187_test_emb/20856_train_8187_test_10_frames/test_labels_embedding.npy'